<a href="https://colab.research.google.com/github/Athugodage/VK_NLP/blob/main/vk_nlp_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install razdel emoji translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=aff511bb573a04c0570eaba84cabf580571927d1d56dbefa8ba34ee752a2d97a
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
  Created wheel f

In [ ]:
import pandas as pd

df = pd.read_csv('/content/posts.csv', encoding='utf8', delimiter=',')

In [ ]:

def clean(text):
  text =  re.sub('[http|www].*', '', text)
  text = re.sub('[#|@][А-яЁёA-z]', '', text)

  return text

df['text'] = df['text'].apply(clean)

In [ ]:
df['text'].iloc[19]

' Лучший детский сад Башкирии : муниципальный этап народного голосования Совместно с Министерством образования и науки РБ запускаем муниципальный этап голосования по лучшему Детскому саду Башкирии по мнению жителей региона. Показать ещё Муниципальный этап голосования пройдёт с 21 июня по 18 июля 2023 года в сообществе Башкирия Online. Региональный этап голосования( участники — победители муниципального этапа голосования + лучшие по числу голосов среди занявших 2 и 3 места) пройдёт с 19 июля по 18 августа 2023 года в сообществе ИА" Башинформ". Подведение итогов конкурса состоится в августе 2023 года. Торжественное награждение победителей состоится 27 сентября - в День работников дошкольного образования. Чтобы проголосовать, перейдите по ссылке своего муниципалитета и выберите участника : ГО г. Агидель : '

In [ ]:
from razdel import tokenize
from tqdm import tqdm
import re

tqdm.pandas()

def check_token(word):

  m  = re.search('[а-яё][А-ЯЁ]', word)
  if m:
    word = word[:m.span()[0]+1]+'. '+word[m.span()[0]+1:]

  return word

def text2tokens(text):
  signs = ('!', '?', ',', '.', '"', ')', '(',)
  tokens = ''
  tokenized = list(tokenize(str(text)))

  for token in tokenized:
    token = check_token(token.text)
    if any(sign in token for sign in signs) == False:
      tokens += ' '

    tokens += token

  return tokens


df['text'] = df['text'].progress_apply(text2tokens)


100%|██████████| 84672/84672 [02:29<00:00, 568.06it/s]


In [ ]:
import emoji
from translate import Translator

translator= Translator(to_lang="ru")

def emoji2text(text, emoji, translator):

  demojized = emoji.demojize(text)

  while re.search(':[A-z]+:', text):
    f = re.search(':[A-z]+:', text)

    emoji = f.group(0)
    print(emoji)
    translated = translator.translate(str(emoji))

    text = text[:f.span()[0]] + translated + text[f.span()[1]:]

  return text


df['text'] = df['text'].progress_apply(lambda x: emoji2text(x, emoji, translator))

100%|██████████| 84672/84672 [01:27<00:00, 965.20it/s]


In [ ]:
df['text'].iloc[19]

' Лучший детский сад Башкирии : муниципальный этап народного голосования Совместно с Министерством образования и науки РБ запускаем муниципальный этап голосования по лучшему Детскому саду Башкирии по мнению жителей региона. Показать ещё Муниципальный этап голосования пройдёт с 21 июня по 18 июля 2023 года в сообществе Башкирия Online. Региональный этап голосования( участники — победители муниципального этапа голосования + лучшие по числу голосов среди занявших 2 и 3 места) пройдёт с 19 июля по 18 августа 2023 года в сообществе ИА" Башинформ". Подведение итогов конкурса состоится в августе 2023 года. Торжественное награждение победителей состоится 27 сентября - в День работников дошкольного образования. Чтобы проголосовать, перейдите по ссылке своего муниципалитета и выберите участника : ГО г. Агидель :'